In [2]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import nltk

In [4]:
from sklearn.model_selection import train_test_split

def load_social_chem101():
    a = pd.read_csv("data/social-chem-101/social-chem-101.v1.0.tsv", sep="\t")
    return a

social_chem = load_social_chem101()
#social_chem = social_chem[social_chem["split"] == "train"]
social_chem = social_chem.dropna(subset=["rot-categorization", "rot-judgment", "action", "rot-agree"])
social_chem = social_chem[social_chem["rot-agree"] >= 3.0]
social_chem = social_chem[social_chem["rot-bad"] == 0]
social_chem = social_chem[social_chem["rot-categorization"].apply(lambda x: "morality-ethics" in x or "social-norms" in x)]
social_chem = social_chem[social_chem["rot-judgment"].apply(lambda x: "{" not in x)]
social_chem = social_chem[social_chem.apply(lambda x: max(len(x["rot"]), len(x["action"]) + len(x["rot-judgment"])) <= 128, axis=1)]
social_chem = social_chem[["action", "rot-judgment", "rot"]].groupby("rot", as_index=False).nth(0)

train, dev = train_test_split(social_chem, test_size=0.1, random_state=42)

In [5]:
# Taken from Moral Stories Github:
# https://github.com/demelin/moral_stories/blob/master/experiments/utils.py

import sacrebleu
from sacrerouge.metrics import Rouge

def compute_gen_metrics(preds, targets):
    """ Aggregates generation metrics. """
    assert len(preds) == len(targets)
    return {'BLEU-4': compute_bleu(preds, targets),
            'ROUGE-L': compute_rouge(preds, targets)}    

def compute_bleu(preds, targets):
    """ Computes corpus-level BLEU for the generated sequences. """
    targets = [targets]
    bleu = sacrebleu.corpus_bleu(preds, targets)
    return bleu.score

def compute_rouge(preds, targets):
    """ Computes ROUGE-L for the generated sequences. """
    rouge = Rouge(compute_rouge_l=True)
    rouge_out = rouge.evaluate(preds, [[tgt] for tgt in targets])
    return rouge_out[0]['rouge-l']['f1']

In [10]:
preds = dev["rot-judgment"] + " " + dev["action"]
compute_gen_metrics(preds.to_list(), dev["rot"].to_list())

{'BLEU-4': 56.47984613335201, 'ROUGE-L': 89.408}